In [6]:
import pandas as pd
import jiwer
import re

In [10]:
patt_newvid = re.compile(r"\-+\s+NEW VIDEO\s+\-+")

In [32]:
sys_ts = []

sys_text = ''

with open('../system_outputs/baselines/wav2vec/harriet_20211019.txt','r') as sysin:
    for l in sysin.readlines():
        
        if re.fullmatch(patt_newvid,l.strip()):
            sys_ts.append(sys_text.strip())
            sys_text = ''
            
        else:
            sys_text = " ".join([sys_text,l.strip()])
        
    if len(sys_text.strip()):
        sys_ts.append(sys_text.strip())
sys_ts

["harriet edith prosser was when i joined the friends that was my name harriet where are you from and why did you decide you want it to be a red well i have had other people ask me that and i think myself because i was open haired a pale skinned and freckle faced i didn't think that the e t s would suit me the uniform you know and i thought that the navy blue wood and i think that's the only reason because we didn't live by the sea we were lived in land you know farming stock where are you from aberdeenshire so you say what what what impact did the war have on you as a young girl living in abiding well am i was still at school of course i went to inverurie academy and i was still at the academy and i remember am there used to be a reconisence plan come over every night you could almost set your watch by it eight clockish and you i knew we all knew that the engine had a different noise to it you know so you could tell that it was a german plane and then one night it was much more noisy 

In [21]:
human_df = pd.read_csv('../transcripts/ingested/Harriet Wright.tsv', delimiter='\t', index_col=0)

In [28]:
hum_ts = []

hum_text = ''

for l in human_df.Transcript:
    if l.strip() == "New Film":
        if len(hum_text.strip()):
            hum_ts.append(hum_text.strip())
        hum_text = ''
            
    else:
        hum_text = " ".join([hum_text,l.strip()])
        
if len(hum_text.strip()):
    hum_ts.append(hum_text.strip())
            
hum_ts

["Harriet Edith Frosser - was, when I joined the wrens- that was my name. Harriet where-where are you from and why did you decide you wanted to be a WREN? oh, well I've had other people ask me that and i think myself because I was auburn haired, um, pale skinned and freckly faced, I didn't think that um the ATS would suit me- the uniform you know (laughs). And I though that the navy blue would. and I think that's the only reason. Really? Because we didn't live by the sea, we were- lived inland you know. Farming stock And where are you from? Aberdeenshire So just tell me what, what impact did the war have on you as a young girl living in Aberdeenshire? Well, um, I was still at school of course. I went to (unclear) academy and I was still at the academy and I remember, um, they used to be a reconnaissance plane come over every night you could almost set your watch by it, 8 o'clock ish and you knew, I, I knew, we all knew that the engine had a different noise to it you know. So you could 

In [34]:
assert len(sys_ts) == len(hum_ts)

In [69]:
# Applied to both human and system transcripts
transformation = jiwer.Compose([
    jiwer.Strip(),
    # Similar to jiwer.RemoveKaldiNonWords() but also removing parentheticals e.g. "(unclear)"
    # NB: need non-greedy match in the middle!
    jiwer.SubstituteRegexes({r"[\[{\(\<].*?[\]}\)\>]": r" ",}), 
    jiwer.RemovePunctuation(),
    jiwer.RemoveWhiteSpace(replace_by_space=True),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemoveEmptyStrings(),
    jiwer.ToLowerCase(),
    jiwer.Strip(),
]) 

In [70]:
# Jiwer metrics calculated at 'sentence' level; here, each sentence is the content of a single video
# NB: if the list lengths differ, everything is concatenated and then compared - so for Frank Wilson we'd be 
#  applying to the complete interview.
mlist = []

for h,s in zip(hum_ts,sys_ts):
    measures = jiwer.compute_measures(h, s,
                truth_transform=transformation, 
                hypothesis_transform=transformation)
    mlist.append(measures)

In [71]:
mlist

[{'wer': 0.12116504854368933,
  'mer': 0.113578449217328,
  'wil': 0.13856653554603937,
  'wip': 0.8614334644539606,
  'hits': 2435,
  'substitutions': 66,
  'deletions': 74,
  'insertions': 172},
 {'wer': 0.08390316022916282,
  'mer': 0.08133285560730921,
  'wil': 0.10599684587673242,
  'wip': 0.8940031541232676,
  'hits': 5128,
  'substitutions': 137,
  'deletions': 146,
  'insertions': 171},
 {'wer': 0.1353009935710111,
  'mer': 0.12954672635702294,
  'wil': 0.15700015939641887,
  'wip': 0.8429998406035811,
  'hits': 3111,
  'substitutions': 92,
  'deletions': 219,
  'insertions': 152},
 {'wer': 0.13143428285857073,
  'mer': 0.1259880239520958,
  'wil': 0.15917698214324738,
  'wip': 0.8408230178567526,
  'hits': 3649,
  'substitutions': 135,
  'deletions': 218,
  'insertions': 173},
 {'wer': 0.1693600517129929,
  'mer': 0.16129694233531705,
  'wil': 0.21655824179576388,
  'wip': 0.7834417582042361,
  'hits': 4087,
  'substitutions': 275,
  'deletions': 279,
  'insertions': 232},
 {'